In [8]:
from langchain_ollama import ChatOllama
from langgraph.prebuilt import create_react_agent
from PDF_Tool import pdf_tool
from CSV_Tool import drone_result_tool
from langchain_core.messages import HumanMessage
from langgraph.checkpoint.memory import MemorySaver
from langchain.prompts import PromptTemplate

In [9]:
# Define the PromptTemplate
prompt = PromptTemplate(
    template="""
    You are an AI agent responsible for choosing the right tool to answer the user's query and dont use interent or online search. Below is the conversation so far:

    {messages}

    Based on the user's latest query, decide which tool to use:
    
    - If the query is related to Bandar, use the `pdf_tool` to extract relevant information from a PDF document.
    - If the query is related to Drones, use the `csv_tool` to extract relevant information from a CSV file.
    
    after using the right tool answer based on information that you got from tool.
    """,
    input_variables=["messages"]
)

In [10]:
# Define memory to remember the chat history
memory = MemorySaver()
# Load the LLM model
llm = ChatOllama(model='mistral')
# Define tools
tools = [pdf_tool, drone_result_tool]

In [11]:
# Create the react agent
agent = create_react_agent(
    tools=tools,
    model=llm,
    state_modifier=prompt,
    checkpointer=memory
)

In [12]:
# Configuration
config = {"configurable": {"thread_id": "1"}}

In [13]:
# User input
user_input = "What is the name of image that have longest time?"

# Prepare the input messages
input_messages = [HumanMessage(user_input)]

# Run the agent
response = agent.invoke({"messages": input_messages}, config=config)

# Print the response
ai_response =response["messages"][-1].pretty_print()

================================== Ai Message ==================================

 Since the user's latest query doesn't seem to be directly related to either Bandar or Drones, I will first process the data to find any potential connections with drones and then use the `csv_tool` to extract relevant information if necessary.

In this case, let's analyze the data for possible drone-related entries. From the data provided, there are multiple document ids (e.g., '62ebcbbc-d0f6-4b87-9e5e-471b17eba89b', '9315aa50-bc16-4d14-b158-1a218834a427', etc.) that appear to be related to drones as they contain terms like 'drone', 'CSV file', or are named after different types of drones (e.g., '20' or '29' which could potentially refer to model numbers).

To proceed, I will use the `csv_tool` and read these identified files to extract any useful information related to the user's query. The specific method to process and analyze the extracted data will depend on the structure of the CSV files and the na

In [9]:
import pandas as pd
# Load documents with Pandas
df = pd.read_excel("./data/DroneResult.xlsx")

In [10]:
df.columns

Index(['image_name', 'db_name', 'length', 'score', 'distance', 'time'], dtype='object')

In [ ]:
from langchain_ollama import ChatOllama
from langgraph.prebuilt import create_react_agent
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from PDF_Tool import pdf_tool
from CSV_Tool import csv_tool
from langchain_core.messages import HumanMessage
from langgraph.checkpoint.memory import MemorySaver

# Define the PromptTemplate
prompt = PromptTemplate(
    template="""
    You are an AI agent responsible for choosing the right tool to answer the user's query and dont use interent or online search. Below is the conversation so far:

    {messages}

    Based on the user's latest query, decide which tool to use:
    
    - If the query is related to Bandar, use the `pdf_tool` to extract relevant information from a PDF document.
    - If the query is related to Drones, use the `csv_tool` to extract relevant information from a CSV file.
    
    after using the right tool answer based on information that you got from tool.
    """,
    input_variables=["messages"]
)

# Load the LLM model
llm = ChatOllama(model='mistral')

# Define tools
tools = [pdf_tool, csv_tool]

# Connect the LLM with the prompt
llm_prompt = LLMChain(llm=llm, prompt=prompt)

# Define memory to remember the chat history
memory = MemorySaver()

# Create the react agent
agent = create_react_agent(
    tools=tools,
    model=llm,
    state_modifier=prompt,
    checkpointer=memory
)

# Configuration
config = {"configurable": {"thread_id": "1"}}

# User input
user_input = "What is bandar major?"

# Prepare the input messages
input_messages = [HumanMessage(user_input)]

# Run the agent
response = agent.invoke({"messages": input_messages}, config=config)

# Print the response
response["messages"][-1].pretty_print()


# Test CSV loader and query 

In [39]:
import pandas as pd
df = pd.read_csv("./data/DroneResult.csv")
df

,image_name,db_name,length,score,distance,time
0,0,10_lIC11X,107,33462.79283,536.676820,1.781866
1,1,11_q2Syp2,111,35161.74298,556.865957,1.781033
2,2,12_EzaSzn,112,34807.45805,482.659030,1.785171
3,3,13_NTftM5,123,38666.17493,500.848725,1.761350
4,4,14_z9YBU1,114,35758.66893,518.708175,1.782434
5,5,15_ywjA7L,131,41412.45232,453.598666,1.781310
6,6,16_IVh1MU,115,36132.38737,463.199200,1.793880
7,7,17_JMFshs,117,36732.36717,474.067247,1.803041
8,8,18_26bPIl,129,40823.18974,412.248950,1.959241
9,9,19_4VNhNR,128,41027.73059,435.247544,1.989626


In [42]:
from langchain_ollama import OllamaEmbeddings
embed = OllamaEmbeddings(model="llama3")

In [45]:
documents=[]
for _, row in df.iterrows():
    document = " | ".join(f"{col}: {row[col]}" for col in df.columns)
    documents.append(document)

In [49]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
# Step 3: Split the data into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
docs = text_splitter.create_documents(documents)

In [51]:
from langchain.vectorstores import FAISS
vector_store = FAISS.from_documents(docs, embed)

In [71]:
# Step 5: Define the Tool
def query_vector_store(query):
    """
    Query the vector store to retrieve similar chunks based on the input query.
    """
    return vector_store.similarity_search(query, k=23)

In [72]:
from langchain.agents import Tool
# Step 5: Define the Tool
drone_result_tool = Tool(
    name="Drone Result Query Tool",
    func=query_vector_store,
    description=(
        "This tool answers questions based on drone result data from the dataset. "
        "The dataset includes columns such as 'image_name', 'db_name', 'length', 'score', 'distance', and 'time'. "
        "based on the query start search in datset and give the final result"
    )
)

In [73]:
# Define a query
query = "What is image name of longest flight?"

# Use the tool to get the results
results = drone_result_tool.func(query)

results

[Document(id='0224755f-3ad4-4452-ab80-68af88535db1', metadata={}, page_content='image_name: 0 | db_name: 10_lIC11X | length: 107 | score: 33462.79283 | distance: 536.67682 | time: 1.781866312'),
 Document(id='7456dd80-a8fb-4922-aa1c-99d50380c24e', metadata={}, page_content='image_name: 3 | db_name: 13_NTftM5 | length: 123 | score: 38666.17493 | distance: 500.8487254 | time: 1.761350393'),
 Document(id='2dfe3022-a662-4417-a60d-1ed43df8b17e', metadata={}, page_content='image_name: 11 | db_name: 20_eH2pdV | length: 126 | score: 40199.26414 | distance: 436.8484482 | time: 1.987559795'),
 Document(id='4cdbe269-f355-4760-a97c-17437d7613bd', metadata={}, page_content='image_name: 21 | db_name: 2_6qTgqL | length: 117 | score: 38105.10632 | distance: 605.1321264 | time: 1.957442045'),
 Document(id='880a2e91-1628-4052-9630-b6c1458fc54e', metadata={}, page_content='image_name: 22 | db_name: 30_eH2hsV | length: 116 | score: 36951.31621 | distance: 267.9790644 | time: 1.993321657'),
 Document(id='5

# Another CSV tool

In [14]:
import pandas as pd 
df = pd.read_csv("./data/DroneResult.csv")
df

,image_name,db_name,length,score,distance,time
0,0,10_lIC11X,107,33462.79283,536.676820,1.781866
1,1,11_q2Syp2,111,35161.74298,556.865957,1.781033
2,2,12_EzaSzn,112,34807.45805,482.659030,1.785171
3,3,13_NTftM5,123,38666.17493,500.848725,1.761350
4,4,14_z9YBU1,114,35758.66893,518.708175,1.782434
5,5,15_ywjA7L,131,41412.45232,453.598666,1.781310
6,6,16_IVh1MU,115,36132.38737,463.199200,1.793880
7,7,17_JMFshs,117,36732.36717,474.067247,1.803041
8,8,18_26bPIl,129,40823.18974,412.248950,1.959241
9,9,19_4VNhNR,128,41027.73059,435.247544,1.989626


In [26]:
from langchain_ollama import OllamaLLM 
model = OllamaLLM(model="llama3")

In [27]:
from langchain_experimental.agents import create_csv_agent
csv_agent = create_csv_agent(
    model,
    './data/DroneResult.csv',
    verbose =True,
    allow_dangerous_code =True
)

In [28]:
config = {"configurable": {"thread_id": "4"}}

In [31]:
csv_agent.run("What is image name of longest time")

C:\Users\B.Almutairi\AppData\Local\Temp\ipykernel_19508\2113740421.py:1: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  csv_agent.run("What is image name of longest time")




> Entering new AgentExecutor chain...
Thought: I need to find the row with the maximum time value.

Action: Use `df.loc[]` to select a row based on the condition that the time column has the maximum value.

Action Input: `df.loc[df['time'].idxmax()]`
Use `df.loc[]` to select a row based on the condition that the time column has the maximum value. is not a valid tool, try one of [python_repl_ast].Let's execute the action:

Action: `print(df.loc[df['time'].idxmax()])`

Action Input: `df.loc[df['time'].idxmax()]`
`print(df.loc[df['time'].idxmax()])` is not a valid tool, try one of [python_repl_ast].I can answer the question!

Question: What is image name of longest time
Thought: I need to find the row with the maximum time value.
Action: python_repl_ast
Action Input: `print(df.loc[df['time'].idxmax()])`image_name             22
db_name         30_eH2hsV
length                116
score         36951.31621
distance       267.979064
time             1.993322
Name: 22, dtype: object
I now k

'The image name of the longest time is 22.'